<a href="https://colab.research.google.com/github/juliasarun/skills-salary-analysis/blob/main/notebooks/skill_salary_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>